# Libraries

In [1]:
# LIBRARIES #

import numpy as np

from scipy import optimize
from scipy.interpolate import CubicSpline
from scipy.signal import wiener

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar, Plot, CustomJS, ColumnDataSource, Rect
from bokeh.layouts import row, gridplot, column
from bokeh.models.widgets import Slider, Button
from bokeh.events import ButtonClick

import SimpleITK as sitk

import time


output_notebook()

Loading BokehJS ...

In [2]:
import warnings

ignoreW = True
if ignoreW :
    warnings.filterwarnings('ignore')

# Functions

In [3]:
# READS THE IMAGE AND CONVERTS IT TO RGBA IMG FOR DISPLAY #

def readImgAndConvertToRGBA(filename):

    # reads the image using simpleITK:
    img = sitk.ReadImage(filename)
    sizex = img.GetWidth()
    sizey = img.GetHeight()
    array = sitk.GetArrayFromImage(img)
    
    # creates a new rgba img and copy the tiff values in it
    rgba = np.empty((sizey,sizex), dtype=np.uint32)
    view = rgba.view(dtype=np.uint8).reshape((sizey, sizex, 4))
    view[:,:,0] = array[:,:,0]/65535.0*255.0
    view[:,:,1] = array[:,:,1]/65535.0*255.0
    view[:,:,2] = array[:,:,2]/65535.0*255.0
    view[:,:,3] = 255

    return array, rgba, sizex, sizey

In [4]:
# CONVERTS THE IMG TO RGBA FOR DISPLAY #

def convertToRGBA(array, sizex, sizey):

    # creates a new rgba img and copy the tiff values in it
    rgba = np.empty((sizey,sizex), dtype=np.uint32)
    view = rgba.view(dtype=np.uint8).reshape((sizey, sizex, 4))
    view[:,:,0] = array[:,:,0]/65535.0*255.0
    view[:,:,1] = array[:,:,1]/65535.0*255.0
    view[:,:,2] = array[:,:,2]/65535.0*255.0
    view[:,:,3] = 255

    return rgba

In [5]:
# READS ALL IMAGES IN THE PATH AND RETURNS THE MEDIAN IMAGE #

def medianImage(path, filename, firstNb, fileExtension, nbOfImgs):

    img = sitk.ReadImage(path+filename+str(firstNb)+fileExtension)
    sizex = img.GetWidth()
    sizey = img.GetHeight()
    size = (sitk.GetArrayFromImage(img).shape[0], 
            sitk.GetArrayFromImage(img).shape[1], 
            sitk.GetArrayFromImage(img).shape[2], 
            nbOfImgs)
    imgs = np.zeros(size)
    for i in range(nbOfImgs):
        img = sitk.ReadImage(path+filename+str(firstNb+i)+fileExtension)
        imgs[:,:,:,i] = sitk.GetArrayFromImage(img)

    medianImg = np.median(imgs, axis=3)

    print('Median Image Generated!')
    print('Image size:', sizex, 'x', sizey, 'x 3 layers (R,G,B)')
    print('')
    
    return medianImg, sizex, sizey

In [6]:
# PRINT MEAN GREY VALUES:

def printGreyValues(rgbvalues):
    for i in range(len(rgbvalues)):
        print('Dose ', str(i), ':', rgbvalues[i][0], 'Gy') 
        print('  - red value:', rgbvalues[i][1])
        print('  - green value:', rgbvalues[i][2])
        print('  - blue value:', rgbvalues[i][3])
        print('')
    

In [7]:
# PLOTS IMAGE AND PROFILES WITH SCROLLERS #
# BUT MAYBE MEMORY PROBLEMS FOR LARGE IMAGES#
# @params:
#   img: array img to display
#   sizex: size of the img in x
#   sizey: size of the img in y
#   imgPlotWidth: width of the image plot

def displayImage(img, sizex, sizey, imgPlotWidth):

    # Displays the dose image (p1):

    maxdose = int(np.amax(img))

    color_mapper = LinearColorMapper(palette="Viridis256", low=0, high=maxdose)

    color_bar = ColorBar(color_mapper=color_mapper, ticker=BasicTicker(),
                     label_standoff=12, border_line_color=None, location=(0,0),
                     title='Dose cGy')

    p1 = figure(plot_width=int(imgPlotWidth*1.1), plot_height=int(imgPlotWidth*sizey/sizex), 
                    x_range=(0,sizex), y_range=(0,sizey), 
                    title="Dose image", toolbar_location="above")

    l1_source = ColumnDataSource(data=dict(x=[0,int(sizex)], y=[int(sizey/2),int(sizey/2)]))
    l2_source = ColumnDataSource(data=dict(x=[int(sizex/2), int(sizex/2)], y=[0, int(sizey)]))

    p1.image(image=[img], x=[0], y=[0], dw=[sizex], dh=[sizey], color_mapper=color_mapper)
    p1.line('x', 'y', source=l1_source, line_width=2, line_color=(255, 255, 255, 0.7))
    p1.line('x', 'y', source= l2_source, line_width=2, line_color=(255, 255, 255, 0.7))

    p1.add_layout(color_bar, 'right')


    # Displays the zoomed window:
    p1b_source = ColumnDataSource({'x': [], 'y': [], 'width': [], 'height': []})

    jscode="""
        var data = source.data;
        var start = cb_obj.start;
        var end = cb_obj.end;
        data['%s'] = [start + (end - start) / 2];
        data['%s'] = [end - start];
        source.change.emit();
    """

    p1.x_range.callback = CustomJS(args=dict(source=p1b_source), code=jscode % ('x', 'width'))
    p1.y_range.callback = CustomJS(args=dict(source=p1b_source), code=jscode % ('y', 'height'))

    p1b = figure(title='Zoom Window', plot_width=int(imgPlotWidth*0.5), 
                     plot_height=int(imgPlotWidth*0.5*sizey/sizex), 
                     x_range=(0,sizex), y_range=(0,sizey), tools='')
    p1b.image(image=[img], x=[0], y=[0], dw=[sizex], dh=[sizey], color_mapper=color_mapper)
    rect = Rect(x='x', y='y', width='width', height='height', fill_alpha=0.1,
                    line_color='white', fill_color='white')
    p1b.add_glyph(p1b_source, rect)

    p1b.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    p1b.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
    p1b.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
    p1b.yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks
    p1b.xaxis.major_label_text_color = None  # turn off x-axis tick labels leaving space
    p1b.yaxis.major_label_text_color = None  # turn off y-axis tick labels leaving space 

    # Displays profiles:
    maxdose_x = np.amax(img[:,:])
    p2_source = ColumnDataSource(data=dict(x=np.arange(0, sizex, 1), y=img[int(sizey/2)]))
    p2 = figure(plot_width=480, plot_height=300, x_range=(0,sizex), y_range=(0,maxdose_x), 
                    title="Horizontal dose profile", toolbar_location="above")
    p2.line('x', 'y', source=p2_source, line_color="#2690d4", line_width=3, line_alpha=1.0)

    p3 = figure(plot_width=480, plot_height=300, x_range=(0,sizey), y_range=(0,maxdose_x), 
                    title="Vertical dose profile", toolbar_location="above")
    p3_source = ColumnDataSource(data=dict(x=np.arange(0, sizey, 1), y=img[:,int(sizex/2)]))
    p3.line('x', 'y', source=p3_source, line_color="#2690d4", line_width=3, line_alpha=1.0)


    # Callback functions called when the sliders are changed
    callback1 = CustomJS(args=dict(source1=l1_source, source2=p2_source), code="""
        var f = horizontalPosSlider.value;

        var data1 = source1.data;
        var y1 = data1['y'];
        y1[0] = f;
        y1[1] = f;
        source1.change.emit();

        var data2 = source2.data;
        var x2 = data2['x'];
        var y2 = data2['y'];
        for (i = 0; i < x2.length; i++) {
            y2[i] = img[f][i];
        }    
        source2.change.emit();
    """)


    callback2 = CustomJS(args=dict(source1=l2_source, source2=p3_source), code="""
        var f = verticalPosSlider.value;

        var data1 = source1.data;
        var x1 = data1['x'];
        x1[0] = f;
        x1[1] = f;
        source1.change.emit();
    
        var data2 = source2.data;
        var x2 = data2['x'];
        var y2 = data2['y'];
        for (i = 0; i < x2.length; i++) {
            y2[i] = img[i][f];
        }    
        source2.change.emit();
    """)

    # Button Callbacks:
    saveCallback = CustomJS(code="""
        console.log("hello");
    """)


    # plotting inline:
    slider1 = Slider(start=0, end=sizey, value=int(sizey/2), step=1, title="Horizontal line position", callback=callback1)
    callback1.args["horizontalPosSlider"] = slider1
    callback1.args["img"] = img

    slider2 = Slider(start=0, end=sizex, value=int(sizex/2), step=1, title="Vertical line position", callback=callback2)
    callback2.args["verticalPosSlider"] = slider2
    callback2.args["img"] = img

    crop_button = Button(label="Crop Image", button_type="default")

    save_button = Button(label="Save Image To TIFF", button_type="default", callback=saveCallback)
    saveCallback.args["img"] = img
    saveCallback.args["folder"] = m_path

    # Organizing the graphs:
    grid = gridplot([[p1,column(p1b,crop_button,save_button)],[slider1,slider2],[p2, p3]])

    show(grid)

In [8]:
# PLOTS IMAGE AND ONE X PROFILE #
# @params:
#   img: array img to display
#   sizex: size of the img in x
#   sizey: size of the img in y
#   linePos: position of the line profile
#   dSum: lines to use for the mean profile are the linePos +- dSum
#   imgPlotWidth: width of the image plot

def simpleImageDisplay(img, sizex, sizey, linePos, dSum=1, imgPlotWidth=500, LinePlotHeight = 300, minvalue=0, maxv=0):

    
    # Main image dose display:
    if (maxv==0): maxvalue = np.amax(img)
    else: maxvalue = maxv

    color_mapper = LinearColorMapper(palette="Viridis256", low=minvalue, high=maxvalue)
    
    color_bar = ColorBar(color_mapper=color_mapper, ticker=BasicTicker(),
                    label_standoff=12, border_line_color=None, location=(0,0),
                    title='')
    
    p1 = figure(plot_width=int(imgPlotWidth*1.1), plot_height=int(imgPlotWidth*sizey/sizex), 
                    x_range=(0,sizex), y_range=(0,sizey), 
                    title="Dose image", toolbar_location="above")
    
    p1.image(image=[img], x=[0], y=[0], dw=[sizex], dh=[sizey], color_mapper=color_mapper)
    p1.line(x=[0,sizex-1], y=[linePos, linePos], line_width=2, line_color="white")
    p1.rect(x=int(sizex/2), y=linePos, width=sizex, height=dSum*2+1, fill_color="white", alpha=0.2) 
    
    p1.add_layout( color_bar, 'right')
    
    # Line profile display:
    yLineDose = np.sum(img[linePos-dSum:linePos+dSum+1,:]/(2*dSum+1), axis=0)
    maxvalue = int(np.amax(yLineDose))
    minvalue = int(np.amin(yLineDose))
    
    p2 = figure(plot_width=int(imgPlotWidth*1.1), plot_height=int(LinePlotHeight), 
#                    x_range=(0,sizex), y_range=(minvalue,int(maxvalue)), 
                    title="Horizontal dose profile", toolbar_location="above")
    
    p2.line(x=range(sizex), y=yLineDose, line_color="#2690d4", line_width=3, line_alpha=1.0)
    
    # Grid:
    grid = gridplot([[p1],[p2]])
    show(grid)

In [9]:
# PLOTS 2 IMAGEs AND ONE X PROFILE FOR THE 2 IMAGES #
# Images must be of the same size
# @params:
#   img1: array img to display
#   img2: array img to display
#   sizex: size of the img in x
#   sizey: size of the img in y
#   linePos: position of the line profile
#   dSum: lines to use for the mean profile are the linePos +- dSum
#   imgPlotWidth: width of the image plot

def doubleImageDisplay(img1, img2, sizex, sizey, linePos, dSum=1, imgPlotWidth=700, LinePlotHeight = 300, identicalScale = True):


    # max dose:
    maxdose1 = np.amax(img1)
    maxdose2 = np.amax(img2)
    if identicalScale: 
        maxdose1 = maxdose2 = max(maxdose1, maxdose2)
    
    # First image display
    color_mapper1 = LinearColorMapper(palette="Viridis256", low=0, high=maxdose1)
    
    color_bar1 = ColorBar(color_mapper=color_mapper1, ticker=BasicTicker(),
                    label_standoff=12, border_line_color=None, location=(0,0),
                    title='Dose cGy')
    
    p1 = figure(plot_width=int(imgPlotWidth/2*1.1), plot_height=int(imgPlotWidth/2*sizey/sizex), 
                    x_range=(0,sizex), y_range=(0,sizey), 
                    title="Dose image 1", toolbar_location="above")
    
    p1.image(image=[img1], x=[0], y=[0], dw=[sizex], dh=[sizey], color_mapper=color_mapper1)
    p1.line(x=[0,sizex-1], y=[linePos, linePos], line_width=2, line_color="white")
    p1.rect(x=int(sizex/2), y=linePos, width=sizex, height=dSum*2+1, fill_color="white", alpha=0.2) 

    
    # Second image display
    color_mapper2 = LinearColorMapper(palette="Viridis256", low=0, high=maxdose2)
    
    color_bar2 = ColorBar(color_mapper=color_mapper2, ticker=BasicTicker(),
                    label_standoff=12, border_line_color=None, location=(0,0),
                    title='Dose cGy')

    p2 = figure(plot_width=int(imgPlotWidth/2*1.1), plot_height=int(imgPlotWidth/2*sizey/sizex), 
                    x_range=(0,sizex), y_range=(0,sizey), 
                    title="Dose image 2", toolbar_location="above")
    
    p2.image(image=[img2], x=[0], y=[0], dw=[sizex], dh=[sizey], color_mapper=color_mapper2)
    p2.line(x=[0,sizex-1], y=[linePos, linePos], line_width=2, line_color="white")
    p2.rect(x=int(sizex/2), y=linePos, width=sizex, height=dSum*2+1, fill_color="white", alpha=0.2) 
    
    
    # Line profiles display:
    yLineDose1 = np.sum(img1[linePos-dSum:linePos+dSum,:]/(2*dSum+1), axis=0)
    maxdose1 = int(np.amax(yLineDose1))
    
    yLineDose2 = np.sum(img2[linePos-dSum:linePos+dSum,:]/(2*dSum+1), axis=0)
    maxdose2 = int(np.amax(yLineDose2))

    p3 = figure(plot_width=int(imgPlotWidth*1.1), plot_height=int(LinePlotHeight), 
                    x_range=(0,sizex), y_range=(0,int(max(maxdose1, maxdose2))+1), 
                    title="Horizontal dose profiles", toolbar_location="above")
    
    p3.line(range(sizex), yLineDose1, line_color="firebrick", line_width=3, legend='Img 1')
    p3.line(range(sizex), yLineDose2, line_color="blue", line_width=3, legend='Img 2')
    
    
    # Grid:
    grid = gridplot([[p1,p2],[p3]])
    show(grid)

# Reads the images

In [10]:
# READS THE IMAGES AND CREATES THE MEDIAN IMAGE:
# <!> ne pas mettre d'accent dans les chemins et noms de fichiers


# Inputs:

m_path = 'G:/Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/scan 24h/'
m_nbOfFiles = 1
m_firstNb = 1
m_filesName = "etalonnage_24h_000"
m_fileExtension = ".tif"

m_dimViewer = 900


# Attention a mettre la liste de dose dans un ordre croissant
m_dose = [0, 25, 50, 100, 200, 300, 400, 500, 600, 800]


# Rectangles de l'image correspondant aux doses énoncées au dessus
rectList = []
rectList.append([1200,700,2000,900])
rectList.append([1200,1000,2000,1050])
rectList.append([1200,1300,2000,1500])
rectList.append([1200,1600,2000,1800])
rectList.append([1200,1900,2000,2100])
rectList.append([1200,2200,2000,2400])
rectList.append([1200,2500,2000,2700])
rectList.append([1200,2800,2000,3000])
rectList.append([1200,3110,2000,3310])
rectList.append([1200,3420,2000,3620])


# Reads the images and computes the mean image:
medianImg, sizex, sizey = medianImage(m_path, m_filesName, m_firstNb, m_fileExtension, m_nbOfFiles)
rgbaimg = convertToRGBA(medianImg, sizex, sizey)


# Plots the areas of interest and calculates the mean rgb grey values in the areas:
p = figure(plot_width=int(m_dimViewer*0.95), plot_height=int(m_dimViewer*0.95*sizey/sizex), 
                    x_range=(0,sizex), y_range=(0,sizey), 
                    title="Gafchromic image", toolbar_location="above")

p.image_rgba(image=[rgbaimg], x=[0], y=[0], dw=[sizex], dh=[sizey])

for i in range(len(rectList)):
    p.rect(x=int(rectList[i][0]+(rectList[i][2]-rectList[i][0])/2), 
           y=int(rectList[i][1]+(rectList[i][3]-rectList[i][1])/2), 
           width=rectList[i][2]-rectList[i][0],
           height=rectList[i][3]-rectList[i][1], 
           fill_color="#0096ff", 
           alpha=0.2)

print("dONE !")
    

#show(p)

Median Image Generated!
Image size: 3098 x 4191 x 3 layers (R,G,B)

dONE !


# Streak artefact correction & Wiener Filter

In [11]:
# Streak artefact correction:


dispGraph = False
dispImg = False


# First the Wiener Filter:
medianImg[:,:,0] = wiener(medianImg[:,:,0])
medianImg[:,:,1] = wiener(medianImg[:,:,1])
medianImg[:,:,2] = wiener(medianImg[:,:,2])


# Mean values of first lines:
firstPixAvg_r = np.sum(medianImg[0:10,:,0] ,axis=0)
firstPixAvg_g = np.sum(medianImg[0:10,:,1] ,axis=0)
firstPixAvg_b = np.sum(medianImg[0:10,:,2],axis=0)

firstPixAvg_r = firstPixAvg_r / np.mean(firstPixAvg_r)
firstPixAvg_g = firstPixAvg_g / np.mean(firstPixAvg_g)
firstPixAvg_b = firstPixAvg_b / np.mean(firstPixAvg_b)


# Mean values of last lines:
lastPixAvg_r = np.sum(medianImg[-9:,:,0] ,axis=0)
lastPixAvg_g = np.sum(medianImg[-9:,:,1] ,axis=0)
lastPixAvg_b = np.sum(medianImg[-9:,:,2],axis=0)

lastPixAvg_r = lastPixAvg_r / np.mean(lastPixAvg_r)
lastPixAvg_g = lastPixAvg_g / np.mean(lastPixAvg_g)
lastPixAvg_b = lastPixAvg_b / np.mean(lastPixAvg_b)


# Streak correction image:
streakCorrImage = np.zeros(medianImg.shape)
x = [0, medianImg.shape[0]-1]

for i in range(medianImg.shape[1]):
    allpix_y = range(medianImg.shape[0])
    y0 = [firstPixAvg_r[i], lastPixAvg_r[i]]
    streakCorrImage[:,i,0] = 1 / np.interp(allpix_y, x, y0)
    
    y1 = [firstPixAvg_g[i], lastPixAvg_g[i]]
    streakCorrImage[:,i,1] = 1 / np.interp(allpix_y, x, y1)

    y2 = [firstPixAvg_b[i], lastPixAvg_b[i]]
    streakCorrImage[:,i,2] = 1 / np.interp(allpix_y, x, y2)
    
    
# Apply streak correction to the image:
medianImg = medianImg * streakCorrImage


print("dONE !")

# disp graphs:
if dispGraph :
    p1 = figure(plot_width=700, plot_height=400, title='first line values', toolbar_location="above")
    p1.line(range(len(firstPixAvg_r)), firstPixAvg_r, line_width=2, line_color='firebrick')
    p1.line(range(len(firstPixAvg_r)), firstPixAvg_g, line_width=2, line_color='green')
    p1.line(range(len(firstPixAvg_r)), firstPixAvg_b, line_width=2, line_color='blue')
    
    p1.line(range(len(firstPixAvg_r)), lastPixAvg_r, line_width=2, line_color='firebrick', line_dash = 'dotted')
    p1.line(range(len(firstPixAvg_r)), lastPixAvg_g, line_width=2, line_color='green', line_dash = 'dotted')
    p1.line(range(len(firstPixAvg_r)), lastPixAvg_b, line_width=2, line_color='blue', line_dash = 'dotted')
    
    show(p1)
    
if dispImg :
    simpleImageDisplay(streakCorrImage[:,:,0], medianImg.shape[1], medianImg.shape[0], \
                       2000, dSum=1, imgPlotWidth=500, LinePlotHeight = 300, minvalue=0.98)

dONE !


# RGB values over dose curves

In [12]:
# CALCULATES RGB VALUES ON THE MEDIAN IMAGE CORRECTED FOR STREAKS:


rvalues, gvalues, bvalues = [], [], []
for i in range(len(rectList)):
    rvalues.append(np.mean(medianImg[rectList[i][1]:rectList[i][3],
                         rectList[i][0]:rectList[i][2], 0]))
    gvalues.append(np.mean(medianImg[rectList[i][1]:rectList[i][3],
                         rectList[i][0]:rectList[i][2], 1]))
    bvalues.append(np.mean(medianImg[rectList[i][1]:rectList[i][3],
                         rectList[i][0]:rectList[i][2], 2]))


print("dONE !")

dONE !


In [14]:
# Plots RGB curves:

p1 = figure(plot_width=700, plot_height=400, title='RGB values', toolbar_location="above")

index_ctrl = m_dose.index(0)

nPVr = []
nPVg = []
nPVb = []
graphDose = []
if index_ctrl == 0 :
    for i in range(1, len(rvalues)):
        nPVr.append(rvalues[index_ctrl]/rvalues[i]-1)
        nPVg.append(gvalues[index_ctrl]/gvalues[i]-1)
        nPVb.append(bvalues[index_ctrl]/bvalues[i]-1)
    nPVdoses = m_dose[1:]
elif index_ctrl == len(rvalues)-1:
    for i in range(0, len(rvalues)-1):
        nPVr.append(rvalues[index_ctrl]/rvalues[i]-1)
        nPVg.append(gvalues[index_ctrl]/gvalues[i]-1)
        nPVb.append(bvalues[index_ctrl]/bvalues[i]-1)
    nPVdoses = m_dose[:index_ctrl]
else:
    print("eRROR: missing ctrl film at first or last place...")
    
p1.line(nPVr, nPVdoses, line_width=2, line_color='firebrick')
p1.line(nPVg, nPVdoses, line_width=2, line_color='green')
p1.line(nPVb, nPVdoses, line_width=2, line_color='blue')

show(column(p1))

# Multilinear Regression:

In [16]:
# Computes the multilinear regression and the nPVrgb:

coefs = np.linalg.lstsq(np.array([[nPVr[i], nPVg[i], nPVb[i]] for i in range(len(nPVr))]), nPVdoses, rcond=None)[0]
print(coefs)

computedDoses = []
for i in range(len(nPVr)):
    computedDoses.append(coefs[0]*nPVr[i] + coefs[1]*nPVg[i] + coefs[2]*nPVb[i])


p1 = figure(plot_width=700, plot_height=400, title='RGB values to dose', toolbar_location="above")    
p1.line(computedDoses, nPVdoses, line_width=2, line_color='darkblue', legend='Computed doses vs. Irradiation doses')

show(p1)

[ -619.5149145   3146.95687925 -4347.08235902]


In [ ]:
# Calculates the dose image of the calibration image:

# Defines the main ROIs
ctrlFilm_rect = [1200,700,2000,900]
toDose_rect = [400,600,2800,3400]


# Finds mean values of R, G and B in the ctrl film
ctrl_r = np.mean(medianImg[ctrlFilm_rect[1]:ctrlFilm_rect[3],
                         ctrlFilm_rect[0]:ctrlFilm_rect[2], 0])
ctrl_g = np.mean(medianImg[ctrlFilm_rect[1]:ctrlFilm_rect[3],
                         ctrlFilm_rect[0]:ctrlFilm_rect[2], 1])
ctrl_b = np.mean(medianImg[ctrlFilm_rect[1]:ctrlFilm_rect[3],
                         ctrlFilm_rect[0]:ctrlFilm_rect[2], 2])


# Computes the dose image using the coefs determined above:
doseImg = coefs[0]*(ctrl_r / medianImg[toDose_rect[1]:toDose_rect[3], toDose_rect[0]:toDose_rect[2],0] - 1) + \
            coefs[1]*(ctrl_g / medianImg[toDose_rect[1]:toDose_rect[3], toDose_rect[0]:toDose_rect[2],1] - 1) + \
            coefs[2]*(ctrl_b / medianImg[toDose_rect[1]:toDose_rect[3], toDose_rect[0]:toDose_rect[2],2] - 1)

dosemax = np.max(m_dose)
doseImg[doseImg>dosemax+50] = 0
doseImg[doseImg<0] = 0

print("Dose image calculated !")


In [ ]:
# Dose image viewer:


# Params
sizex = toDose_rect[2] - toDose_rect[0]
sizey = toDose_rect[3] - toDose_rect[1]
imgPlotWidth = 500

# Viewer with Bokeh (may crash if dose image too large)
#displayImage(doseImg, sizex, sizey, imgPlotWidth)


# Simple Viewer: (can be used to create mean profiles)
simpleImageDisplay(doseImg, sizex, sizey, 2600, dSum=50)


# Fingerprint Correction

In [ ]:
# Fingerprints at the time of calibration:

display = False

# Fingerprints calculation using mean values in the ROIs
Ccal_r = []
Ccal_g = []
Ccal_b = []
for i in range(len(rvalues)):
    Ccal_r.append(rvalues[i] / (rvalues[i] + gvalues[i] + bvalues[i]))
    Ccal_g.append(gvalues[i] / (rvalues[i] + gvalues[i] + bvalues[i]))
    Ccal_b.append(bvalues[i] / (rvalues[i] + gvalues[i] + bvalues[i]))


# Display the values
if display:
    p1 = figure(plot_width=700, plot_height=400, title='RGB values to dose', toolbar_location="above")    
    p1.line(m_dose, Ccal_r, line_width=2, line_color='firebrick', legend='Ccal_r')
    p1.line(m_dose, Ccal_g, line_width=2, line_color='green', legend='Ccal_g')
    p1.line(m_dose, Ccal_b, line_width=2, line_color='darkblue', legend='Ccal_b')

    show(p1)


In [ ]:
# Fingerprints at the time of calibration:

# Function to be used to iterate:
def fingerprintIteration(array, doseimg, ctrl_values, cmeasr, cmeasg, cmeasb, xdose, ccal_r, ccal_g, ccal_b):
    
    # Calculation of the interpolated Ccal for each pixel
    ccalr = np.interp(doseimg, xdose, ccal_r)
    ccalg = np.interp(doseimg, xdose, ccal_g)
    ccalb = np.interp(doseimg, xdose, ccal_b)

    meanCcal = [np.mean(ccalr), np.mean(ccalg), np.mean(ccalb)]

    # Computes correction Factors:
    cr = cmeasr / ccalr
    cg = cmeasg / ccalg
    cb = cmeasb / ccalb


    # New dose image:
    imgCorr = cr*coefs[0]*(ctrl_values[0] / array[:,:,0] - 1) + \
                cg*coefs[1]*(ctrl_values[1] / array[:,:,1] - 1) + \
                cb*coefs[2]*(ctrl_values[2] / array[:,:,2] - 1)

    return imgCorr, meanCcal
    

# time beginning execution
time1 = time.time()


# Part of the input image to convert:
ctrlFilm_rect = [1200,700,2000,900]
toDose_rect = [400,600,2800,3400]


# new array containing only the elements to convert to dose:
arrayToConvert = medianImg[toDose_rect[1]:toDose_rect[3], toDose_rect[0]:toDose_rect[2], :]
sizex = toDose_rect[2] - toDose_rect[0]
sizey = toDose_rect[3] - toDose_rect[1]


# Finds mean values of R, G and B in the ctrl film
ctrl_r = np.mean(medianImg[ctrlFilm_rect[1]:ctrlFilm_rect[3],
                         ctrlFilm_rect[0]:ctrlFilm_rect[2], 0])
ctrl_g = np.mean(medianImg[ctrlFilm_rect[1]:ctrlFilm_rect[3],
                         ctrlFilm_rect[0]:ctrlFilm_rect[2], 1])
ctrl_b = np.mean(medianImg[ctrlFilm_rect[1]:ctrlFilm_rect[3],
                         ctrlFilm_rect[0]:ctrlFilm_rect[2], 2])
ctrl_values = [ctrl_r, ctrl_g, ctrl_b]


# Computes the first dose image using the coefs determined above:
doseImg = coefs[0]*(ctrl_values[0] / arrayToConvert[:,:,0] - 1) + \
            coefs[1]*(ctrl_values[1] / arrayToConvert[:,:,1] - 1) + \
            coefs[2]*(ctrl_values[2] / arrayToConvert[:,:,2] - 1)


# Computes Cmeas values:
Cmeas_r = arrayToConvert[:,:, 0] / (arrayToConvert[:,:, 0] \
            + arrayToConvert[:,:, 1] + arrayToConvert[:,:, 2])
Cmeas_g = arrayToConvert[:,:, 1] / (arrayToConvert[:,:, 0] \
            + arrayToConvert[:,:, 1] + arrayToConvert[:,:, 2])
Cmeas_b = arrayToConvert[:,:, 2] / (arrayToConvert[:,:, 0] \
            + arrayToConvert[:,:, 1] + arrayToConvert[:,:, 2])


# Calculate Ccal and new image dose
doseImgCorr, meanCcal0 = fingerprintIteration(arrayToConvert, doseImg, ctrl_values, \
                                              Cmeas_r, Cmeas_g, Cmeas_b, m_dose, Ccal_r, Ccal_g, Ccal_b)

meanCcalIter = [0,0,0]
meanCcalIter[0] = meanCcal0[0] + 1
meanCcalIter[1] = meanCcal0[1] + 1
meanCcalIter[2] = meanCcal0[2] + 1
i = 0
while abs((meanCcalIter[0]-meanCcal0[0])/meanCcal0[0]) > 0.005 \
            and abs((meanCcalIter[1]-meanCcal0[1])/meanCcal0[1]) > 0.005 \
            and abs((meanCcalIter[2]-meanCcal0[2])/meanCcal0[2]) > 0.005 :
    doseImgCorr, meanCcalIter = fingerprintIteration(arrayToConvert, doseImgCorr, ctrl_values, \
                                                     Cmeas_r, Cmeas_g, Cmeas_b, m_dose, Ccal_r, Ccal_g, Ccal_b)
    i += 1
    print("Iter no:", i)
    print("convergence R:", (meanCcalIter[0]-meanCcal0[0])/meanCcalIter[0]*100, "%")
    print("convergence R:", (meanCcalIter[1]-meanCcal0[1])/meanCcalIter[1]*100, "%")
    print("convergence R:", (meanCcalIter[2]-meanCcal0[2])/meanCcalIter[2]*100, "%")
    print("\n")
    

# Execution time:
time2 = time.time()
print('\n\n(execution time:', np.around(time2-time1, 2), 's)')


In [ ]:
imgPlotWidth = 500
    
#simpleImageDisplay(doseImg, sizex, sizey, 200, dSum=50)
doubleImageDisplay(doseImg, doseImgCorr, sizex, sizey, 1700, dSum=1)

# Saving Calibration data

In [ ]:
# SAVE CALIBRATION DATA:

#  In the case of a bspline fitting:
file1 = open(m_path+"bSpline_data.txt","w+")

file1.write("BSpline fitting data \n\n")
file1.write("R/B value\tdose\n")
for i in range(len(dose)):
    file1.write(str(rbvalues[i])+'\t'+str(dose[i])+'\n')

file1.close()